In [23]:
import os 
import json
import pandas as pd  
import traceback

In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
# my_key = os.getenv("GEMINI_API_KEY")

# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro",
#     api_key = my_key,
#     temperature=0.5
# )

In [26]:
my_key = os.getenv("MISTRAL_API_KEY")

In [27]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    api_key = my_key,
    model="mistral-large-latest",
    temperature=0.5,
    max_retries=2,
)

In [28]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2


In [29]:
RESPONSE_JSON = {
    "quizzes": [
        {
            "question_number": 1,
            "question": "First generated question here based on {text} and {subject}",
            "options": [
                "A) Option A",
                "B) Option B",
                "C) Option C",
                "D) Option D"
            ],
            "correct_answer": "B",
            "difficulty": "Medium",
            "explanation": "Explanation for the correct answer."
        },
        {
            "question_number": 2,
            "question": "Second generated question here based on {text} and {subject}",
            "options": [
                "A) Option A",
                "B) Option B",
                "C) Option C",
                "D) Option D"
            ],
            "correct_answer": "C",
            "difficulty": "Hard",
            "explanation": "Explanation for the correct answer."
        }
    ]
}


In [30]:
TEMPLATE1 = """
You are tasked with creating {number} multiple-choice questions for an educational quiz. 
The topic of each question should be based on {text}. 
The subject is {subject}, and the tone of the questions should be {tone}. 
Ensure that each question effectively tests the user's knowledge, is clear, concise, and does not repeat the answer or options. 
For each question, provide {number} options, only one of which is correct. 
Return all questions and options in the following 
### RESPONSE_JSON 
{response_json}
"""

In [31]:
prompt1 = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE1
)

In [32]:
chain1 = LLMChain(prompt=prompt1, llm=llm, output_key="quiz",verbose=True)

In [33]:
TEMPLATE2 = """
You are tasked with evaluating the quiz based on the subject: {subject}. 
Please review the quiz questions and options provided in the {quiz}. 
Ensure that the questions effectively assess the knowledge of the respondents, are clear and concise, and that the options are distinct from each other. 
Provide feedback on the clarity of the questions and the appropriateness of the options given. 
Make suggestions for any improvements needed for better evaluation.
"""

In [34]:
prompt2 = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

In [35]:
chain2 = LLMChain(prompt=prompt2, llm=llm, output_key="review",verbose=True)

In [36]:
combine_chain = SequentialChain(
    chains = [chain1, chain2],
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables=["quiz","review"],
    verbose=True
)

In [37]:
file_path = r"D:\HP_Laptop\Project\04_GenAI\MCQ_Generator\data.txt"

In [38]:
with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]

Generative AI has uses across a wide range of in

In [39]:
NUMBER = 5
SUBJECT = "Generative AI"
TONE = "Simple"

In [40]:
RES_JSON = json.dumps(RESPONSE_JSON)

In [41]:
response = None  
try:
    with get_openai_callback() as cb:
        response = combine_chain.invoke({
            'text': TEXT, 
            'number': NUMBER,
            'subject': SUBJECT,
            'tone': TONE,
            'response_json': RES_JSON
        })
except Exception as e:
    print(f"An error occurred: {e}")




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are tasked with creating 5 multiple-choice questions for an educational quiz. 
The topic of each question should be based on Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI gen

In [42]:
if response:
    print(response)
else:
    print("No response generated.")

{'text': 'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]\n\nGenerative AI has uses across a wi

In [43]:
print(f"Total Tokens : {cb.total_tokens}")
print(f"Prompt Tokens : {cb.prompt_tokens}")
print(f"Completion Tokens : {cb.completion_tokens}")
print(f"Total Cost : {cb.total_cost}")

Total Tokens : 3168
Prompt Tokens : 1572
Completion Tokens : 1596
Total Cost : 0.0


In [44]:
quiz = response.get('quiz')
print(type(quiz))
quiz

<class 'str'>


'```json\n{\n  "quizzes": [\n    {\n      "question_number": 1,\n      "question": "What is the primary purpose of generative AI?",\n      "options": [\n        "A) To analyze existing data",\n        "B) To create new data based on patterns learned from training data",\n        "C) To perform calculations",\n        "D) To store information",\n        "E) To predict future events"\n      ],\n      "correct_answer": "B",\n      "difficulty": "Easy",\n      "explanation": "Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data."\n    },\n    {\n      "question_number": 2,\n      "question": "Which of the following is an example of a generative AI model?",\n      "options": [\n        "A) Google Search",\n        "B) ChatGPT",\n        "C) Excel",\n        "D) Photoshop",\n        "E) Microsoft Word"\n      ],\n      "correct_answer": "B",\n      "difficulty": "Easy",\n      "explanation": "ChatGPT is a well-known examp

In [45]:
quiz = quiz.replace('```', '').strip()
quiz = quiz.lstrip("json\n").strip()  
quiz


'{\n  "quizzes": [\n    {\n      "question_number": 1,\n      "question": "What is the primary purpose of generative AI?",\n      "options": [\n        "A) To analyze existing data",\n        "B) To create new data based on patterns learned from training data",\n        "C) To perform calculations",\n        "D) To store information",\n        "E) To predict future events"\n      ],\n      "correct_answer": "B",\n      "difficulty": "Easy",\n      "explanation": "Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data."\n    },\n    {\n      "question_number": 2,\n      "question": "Which of the following is an example of a generative AI model?",\n      "options": [\n        "A) Google Search",\n        "B) ChatGPT",\n        "C) Excel",\n        "D) Photoshop",\n        "E) Microsoft Word"\n      ],\n      "correct_answer": "B",\n      "difficulty": "Easy",\n      "explanation": "ChatGPT is a well-known example of a g

In [46]:
quiz_dict = json.loads(quiz)
print(type(quiz_dict))
quiz_dict

<class 'dict'>


{'quizzes': [{'question_number': 1,
   'question': 'What is the primary purpose of generative AI?',
   'options': ['A) To analyze existing data',
    'B) To create new data based on patterns learned from training data',
    'C) To perform calculations',
    'D) To store information',
    'E) To predict future events'],
   'correct_answer': 'B',
   'difficulty': 'Easy',
   'explanation': 'Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.'},
  {'question_number': 2,
   'question': 'Which of the following is an example of a generative AI model?',
   'options': ['A) Google Search',
    'B) ChatGPT',
    'C) Excel',
    'D) Photoshop',
    'E) Microsoft Word'],
   'correct_answer': 'B',
   'difficulty': 'Easy',
   'explanation': 'ChatGPT is a well-known example of a generative AI model that can produce text in response to specific prompts.'},
  {'question_number': 3,
   'question': 'What type of neural network has si

In [47]:
quiz_data = []

for value in quiz_dict['quizzes']:
    mcq = value['question']
    options = " | ".join(value['options'])
    correct = value['correct_answer']
    
    quiz_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct
    })

In [48]:
quiz_data

[{'MCQ': 'What is the primary purpose of generative AI?',
  'Choices': 'A) To analyze existing data | B) To create new data based on patterns learned from training data | C) To perform calculations | D) To store information | E) To predict future events',
  'Correct': 'B'},
 {'MCQ': 'Which of the following is an example of a generative AI model?',
  'Choices': 'A) Google Search | B) ChatGPT | C) Excel | D) Photoshop | E) Microsoft Word',
  'Correct': 'B'},
 {'MCQ': 'What type of neural network has significantly contributed to the advancement of generative AI?',
  'Choices': 'A) Convolutional Neural Networks | B) Recurrent Neural Networks | C) Transformer-based Deep Neural Networks | D) Feedforward Neural Networks | E) Autoencoders',
  'Correct': 'C'},
 {'MCQ': 'Which of the following is a text-to-image AI generation system?',
  'Choices': 'A) ChatGPT | B) Stable Diffusion | C) Copilot | D) LLaMA | E) Sora',
  'Correct': 'B'},
 {'MCQ': 'What is one of the potential concerns about the us

In [49]:
quiz_df = pd.DataFrame(quiz_data)
quiz_df

,MCQ,Choices,Correct
0,What is the primary purpose of generative AI?,A) To analyze existing data | B) To create new...,B
1,Which of the following is an example of a gene...,A) Google Search | B) ChatGPT | C) Excel | D) ...,B
2,What type of neural network has significantly ...,A) Convolutional Neural Networks | B) Recurren...,C
3,Which of the following is a text-to-image AI g...,A) ChatGPT | B) Stable Diffusion | C) Copilot ...,B
4,What is one of the potential concerns about th...,A) Increased job security | B) Improved data p...,C


In [50]:
quiz_df.to_csv("quiz_data.csv",index=False)

In [51]:
from datetime import datetime

datetime.now()

datetime.datetime(2024, 10, 21, 22, 21, 12, 964388)

In [52]:
datetime.now().strftime('%d_%m_%Y_%H_%M_%S')

'21_10_2024_22_21_12'